In [17]:
!pip install keras.tuner


Processing /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6/keras_tuner-1.0.2-cp37-none-any.whl


In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [19]:
fashion_mnist= keras.datasets.fashion_mnist

In [20]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [21]:
train_images= train_images/255.0
test_images=test_images/255.0

In [22]:
train_images[0].shape

(28, 28)

In [23]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [24]:
def build_model(hp):
  model= keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu'    
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [25]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [26]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [27]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 11m 22s]
val_accuracy: 0.9166666865348816

Best val_accuracy So Far: 0.9166666865348816
Total elapsed time: 01h 46m 47s
INFO:tensorflow:Oracle triggered exit


In [30]:
model=tuner_search.get_best_models(num_models=1)[0]

In [31]:
model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f76ff065510>>